In [ ]:
from SoccerNet.Downloader import SoccerNetDownloader
d = SoccerNetDownloader(LocalDirectory="SoccerNet")
d.downloadGames(files=["Labels-v2.json","Labels-cameras.json"], split=["train","valid","test"])

In [ ]:
import json
from moviepy.editor import VideoFileClip
import os 

# Parameters
VIDEO_PATH = "soccer.mp4"
JSON_PATH = "/Users/hebah/Documents/KAUST/STATS/ASRG/sn-spotting/Benchmarks/CALF/inference/outputs/Predictions-v2.json"
OUTPUT_DIR = "clips2/"
BEFORE = 5  # seconds before the event
AFTER = 5   # seconds after the event

# Labels we care about
TARGET_LABELS = {
    "Goal",
    "Penalty",
    "Foul",
    "Shot on target",
    "Red card",
    "Yellow card",
    "Direct free kick",
}

# Make sure output directory exists
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Load predictions
with open(JSON_PATH, "r") as f:
    data = json.load(f)

predictions = data["predictions"]

# Load video
video = VideoFileClip(VIDEO_PATH)

for i, pred in enumerate(predictions):
    label = pred["label"]

    # Skip labels we don't care about
    if label not in TARGET_LABELS:
        continue

    # Get event time in seconds
    event_time = int(pred["position"]) / 1000.0  

    # Define clip boundaries
    start = max(0, event_time - BEFORE)
    end = min(video.duration, event_time + AFTER)
    
    # Cut clip
    subclip = video.subclip(start, end)
    
    # Define output filename
    safe_label = label.replace(" ", "_")
    output_path = f"{OUTPUT_DIR}event_{i+1}_{safe_label}.mp4"
    
    # Save
    subclip.write_videofile(output_path, codec="libx264", audio_codec="aac")

video.close()